In [1]:
def script_method(fn, _rcb=None):
    return fn
def script(obj, optimize=True, _frames_up=0, _rcb=None):
    return obj    
import torch.jit
torch.jit.script_method = script_method 
torch.jit.script = script
import tkinter as tk
import os
from tkinter import filedialog
from tkinter import *
import tensorflow.keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions, preprocess_input
from tensorflow.keras.models import Model
from tkinter import messagebox
from tkinter import simpledialog
from tkinter import messagebox as mb
from sklearn.decomposition import PCA
from scipy.spatial import distance
modelt = tensorflow.keras.applications.VGG16(weights='imagenet', include_top=True)
feat_extractor = Model(inputs=modelt.input, outputs=modelt.get_layer("fc2").output)
import PyPDF2
from PyPDF2 import PdfFileWriter, PdfFileReader
import fitz
import io
import numpy as np
from PIL import ImageTk
from PIL import Image
import cv2
from tkinter import ttk
from PIL import ImageDraw
from PIL import ImageFont
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
import threading


def add_margin(pil_img, top, right, bottom, left, color,thumb_height):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    result = result.resize((int(pil_img.width * thumb_height / pil_img.height), thumb_height))
    return result
def crop_object(image, box):
  im_pil = Image.fromarray(image)  
  x_top_left = box[0]
  y_top_left = box[1]
  x_bottom_right = box[2]
  y_bottom_right = box[3]
  x_center = (x_top_left + x_bottom_right) / 2
  y_center = (y_top_left + y_bottom_right) / 2
  crop_img = im_pil.crop((int(x_top_left), int(y_top_left), int(x_bottom_right), int(y_bottom_right)))
  return crop_img
def extract_image():
    path=filedialog.askopenfilename()
    if len(path) > 0:
        disable_button('disabled')
        tk.messagebox.showinfo(title=None, message="Extraction of images from PDF initiated  ")
        with fitz.open(path) as my_pdf_file:
            for page_number in range(0, len(my_pdf_file)):
                if page_number<=len(my_pdf_file):
                    page = my_pdf_file[page_number-1]
                    pix = page.get_pixmap()  # render page to an image
                    stream = pix.pil_tobytes(format="PNG", optimize=True)
                    stream = io.BytesIO(stream)
                    img = Image.open(stream)
                    img=np.array(img)
                    outputs = predictor(img)
                    MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes = ['cloth']
                    boxes = outputs["instances"].pred_boxes
                    for i in range(0,len(boxes)):
                        box = list(boxes)[i].detach().cpu().numpy()
                        crop_img = crop_object(img, box)
                        os.makedirs(images_path+"\\"+str(os.path.splitext(os.path.basename(path))[0]),exist_ok=True)
                        saved_dir=images_path+"\\"+str(os.path.splitext(os.path.basename(path))[0])
                        crop_img.save(str(saved_dir)+"\\"+str(os.path.splitext(os.path.basename(path))[0])+"_pg"+str(page_number)+"_"+str(i)+".png")
        tk.messagebox.showinfo(title=None, message="Extraction Completed  ")
        analyze()
def get_concatenated_images(src_img,indexes, thumb_height):
    thumbs = []
    lst_names=[]
    for idx in indexes:
        img = image.load_img(images[idx])
        img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
        txt=os.path.basename(images[idx])
        lst_names.append(txt)
        thumbs.append(img)
    return (thumbs,lst_names)
def load_image(path):
    img = image.load_img(path, target_size=modelt.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x
def openNewWindow(PIL_image,lst_names,src_img):
    Main = Toplevel(root)
    Main.title("Result") 
    Main.geometry("1280x720")

    main_frame = Frame(Main)
    main_frame.pack(side=TOP ,fill=BOTH, expand=1)
    
    my_canvas = Canvas(main_frame)
    my_canvas.pack(side=BOTTOM, fill=BOTH, expand=1)

    src_img = ImageTk.PhotoImage(src_img)
    w=Label(main_frame, image=src_img)
    w.pack(side=BOTTOM)
    
    w.photo = src_img
    text = Label(main_frame, text="Selected Image")
    text.pack(side=BOTTOM)
        
    newWindow = Frame(my_canvas)
    
    # Add that New frame To a Window In The Canvas
    my_canvas.create_window((10,100), window=newWindow, anchor="nw")
    
    my_scrollbar = tk.Scrollbar(main_frame, orient="horizontal", command=my_canvas.xview)
    my_scrollbar.pack( side = BOTTOM, fill = X )
    
    my_canvas.configure(xscrollcommand=my_scrollbar.set)
    my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion = my_canvas.bbox("all")))
    my_canvas.update_idletasks()
    
    my_scrollbar.config( command = my_canvas.xview )
    for i in range(len(PIL_image)):
        PIL_image[i] = ImageTk.PhotoImage(PIL_image[i])
        w=Label(newWindow, image=PIL_image[i])
        w.grid(row=0, column=i)
        w.photo = PIL_image[i]
        text1 = Label(newWindow, text=lst_names[i])
        text1.grid(row=1, column=i)    
 
    Main.state('zoomed')
    

def select_image():
    path=filedialog.askopenfilename()
    if len(path) > 0:
        image, x = load_image(path)
        new_features = feat_extractor.predict(x)
        new_pca_features = pca.transform(new_features)[0]
        distances = [ distance.cosine(new_pca_features, feat) for feat in pca_features ]
        idx_closest = sorted(sorted(range(len(distances)), key=lambda k: distances[k])[0:10])  # grab first 5
        results_image,lst_names = get_concatenated_images(image,idx_closest, 200)
        lst=[]
        for idx in idx_closest:
            lst.append(images[idx][images[idx].find('\\'):])
        openNewWindow(results_image,lst_names,image)
cwd= os.getcwd()
def analyze_image():
    disable_button('disabled')
    images_path = str(cwd)+'\\Image_stash'
    image_extensions = ['.jpg', '.png', '.jpeg'] 
    max_num_images = 100000
    global images 
    images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
    tk.messagebox.showinfo(title=None, message="Analyzing  features for %d images" % len(images))
    if max_num_images < len(images):
        images = [images[i] for i in sorted(random.sample(xrange(len(images)), max_num_images))]
    global features 
    features = []
    for i, image_path in enumerate(images):
        img, x = load_image(image_path);
        feat = feat_extractor .predict(x)[0]
        features.append(feat)
    features = np.array(features)
    global pca
    pca = PCA(n_components=len(images))
    pca.fit(features)
    global pca_features 
    pca_features=pca.transform(features)
    tk.messagebox.showinfo(title=None, message="Finished analyzing  features for %d images" % len(images))
    disable_button('normal')
config_file_path = str(cwd)+"\config.yaml"
weights_path = str(cwd)+"\model_final.pth"
model = config_file_path
cfg = get_cfg()
cfg.merge_from_file(config_file_path)
cfg.MODEL.DEVICE='cpu'
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8
predictor = DefaultPredictor(cfg)
def extract():
    t = threading.Thread(target=extract_image)
    t.setDaemon(True)
    t.start()
def select():
    s=threading.Thread(target=select_image)
    s.setDaemon(True)
    s.start()

def analyze():
    a=threading.Thread(target=analyze_image)
    a.setDaemon(True)
    a.start()

def disable_button(state):
    if state=='disabled':
        btn['state'] = 'disabled'
        btn1['state'] = 'disabled'

    if state=='normal':
        btn['state'] = 'normal'
        btn1['state'] = 'normal'

root=tk.Tk()
root.title('Cloth Matcher')
# panelA = None
# panelB = None
global pca,pca_features,features,images,images_path,path,page_number,path,saved_dir,crop_img,results_image,lst_names,results_image
btn = Button(root, text="Search Image", command=select)
btn.pack(side="bottom", fill="both", expand="yes", padx="100", pady="100")
btn1 = Button(root, text="Extract catalogue", command=extract)
btn1.pack(side="bottom", fill="both", expand="yes", padx="100", pady="100")
t = Text(root)
myFrame = Frame(root).place(x=500, y=100)
os.makedirs('Image_stash',exist_ok=True)
images_path = str(cwd)+'\\Image_stash'
image_extensions = ['.jpg', '.png', '.jpeg'] 
max_num_images = 10000
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
if images:
    disable_button("disabled")
    tk.messagebox.showinfo(title=None, message="Analyzing  features for %d images" % len(images))
    if max_num_images < len(images):
        images = [images[i] for i in sorted(random.sample(xrange(len(images)), max_num_images))]
    features = []
    for i, image_path in enumerate(images):
        img, x = load_image(image_path);
        feat = feat_extractor .predict(x)[0]
        features.append(feat)
    features = np.array(features)
    pca = PCA(n_components=len(images))
    pca.fit(features)
    pca_features=pca.transform(features)
    tk.messagebox.showinfo(title=None, message=" Finished analyzing  features for %d images  " % len(images))
    disable_button("normal")
else:
    tk.messagebox.showinfo(title=None, message="No images found in Image Stash , Please extract images from catalogue first ")

root.mainloop()

C:\Users\AkshayAnvekar\anaconda3\envs\pyinstaller\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

